## Verification of Two Model Forecasts vs Observations (Measurements)

Documentation about DMI dat and Scores can be found here:
- https://opendatadocs.dmi.govcloud.dk/APIs/Meteorological_Observation_API
- https://scores.readthedocs.io/en/stable/index.html

Plese define the folling variables to define the data you want to verify. Each model var must have a corresponding obs var after processing.

In [ ]:
path_model_1 = (
    "/users/sadamov/pyprojects/neural-lam/data/danra/single_levels.zarr"
)
path_model_2 = path_model_1  # Using same data for demonstration
url_model = (
    "https://mllam-test-data.s3.eu-north-1.amazonaws.com/single_levels.zarr"
)
vars_model = [
    "u10m",
    "v10m",
    "t2m",
    "pres_seasurface",
    "pres0m",
]

path_obs = "/users/sadamov/pyprojects/neural-lam/data/danra/observations.zarr"
url_obs = "https://dmigw.govcloud.dk/v2/metObs/collections/observation/items"
vars_obs = [
    "wind_speed_past1h",
    "wind_dir_past1h",
    "temp_dry",
    "pressure_at_sea",
    "pressure",
]
api_key = "8dff599e-9a48-46eb-a166-72f2f722645e"

# Chose subset if needed
vars_plot = vars_model

In [2]:
class SynopProcessor:
    def __init__(self):
        self.var_mapping = {
            "temp_dry": "t2m",
            "pressure_at_sea": "pres_seasurface",
            "pressure": "pres0m",
        }

    def calculate_wind_components(
        self, ds, speed_var="wind_speed_past1h", dir_var="wind_dir_past1h"
    ):
        """Calculate u and v wind components from speed and direction."""
        ds = ds.copy()
        ds["u10m"] = ds[speed_var] * np.cos(np.radians(90 - ds[dir_var]))
        ds["v10m"] = ds[speed_var] * np.sin(np.radians(90 - ds[dir_var]))
        ds = ds.drop_vars([speed_var, dir_var])
        return ds

    def rename_variables(self, ds):
        """Rename variables according to mapping."""
        return ds.rename_vars(self.var_mapping)

    def convert_units(self, ds):
        """Convert temperature to Kelvin and pressure to Pa."""
        ds = ds.copy()
        # Convert temperature from Celsius to Kelvin
        if "temp_dry" in ds:
            ds["temp_dry"] = ds["temp_dry"] + 273.15

        # Convert pressure from hPa to Pa
        for pressure_var in ["pressure", "pressure_at_sea"]:
            if pressure_var in ds:
                ds[pressure_var] = ds[pressure_var] * 100
        return ds

    def process_dataset(self, ds):
        """Process the entire dataset with unit conversions, wind calculation and variable renaming."""
        ds = self.convert_units(ds)
        ds = self.calculate_wind_components(ds)
        ds = self.rename_variables(ds)
        return ds

In [3]:
import os

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import xarray as xr
from scipy.interpolate import RBFInterpolator
from scipy.spatial import KDTree
from scipy.stats import gaussian_kde, wasserstein_distance
from scores.continuous import (
    mae,
    mse,
    rmse,
)
from scores.continuous.correlation import pearsonr

/users/sadamov/miniforge3/envs/neural-lam/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


For this example the model data is coming from the AWS bucket from DMI for the DANRA reanalysis.

In [4]:
# If the data is not available locally, download it from the cloud
if not os.path.exists(path_model_1):
    ds_model = xr.open_zarr(url_model)
    chunk_dict = {dim: -1 for dim in ds_model.dims if dim != "time"}
    chunk_dict["time"] = 20
    ds_model = ds_model.chunk(chunk_dict)

    for var in ds_model.variables:
        if "chunks" in ds_model[var].encoding:
            del ds_model[var].encoding["chunks"]

    ds_model.to_zarr(path_model_1, mode="w")
else:
    ds_model = xr.open_zarr(path_model_1)

# Adjust time for both models (simulating different forecast steps)
ds_model["time"] = (
    pd.to_datetime(ds_model["time"].values) + pd.DateOffset(years=30)
).to_numpy()
ds_model_1 = ds_model.isel(time=slice(1, None))[
    vars_model
]  # One timestep offset
ds_model_2 = ds_model.isel(time=slice(3, None))[
    vars_model
]  # Three timesteps offset
ds_model_1 = ds_model_1.assign_coords(
    time=ds_model.isel(time=slice(0, -1)).time
)
ds_model_2 = ds_model_2.assign_coords(
    time=ds_model.isel(time=slice(0, -3)).time
)

ds_model

<xarray.Dataset> Size: 10GB
Dimensions:          (time: 100, y: 589, x: 789)
Coordinates:
    lat              (y, x) float64 4MB dask.array<chunksize=(589, 789), meta=np.ndarray>
    lon              (y, x) float64 4MB dask.array<chunksize=(589, 789), meta=np.ndarray>
  * time             (time) datetime64[ns] 800B 2020-09-01 ... 2020-09-13T09:...
  * x                (x) float64 6kB -1.999e+06 -1.997e+06 ... -2.925e+04
  * y                (y) float64 5kB -6.095e+05 -6.07e+05 ... 8.58e+05 8.605e+05
Data variables: (12/27)
    cape_column      (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    cb_column        (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    ct_column        (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    grpl_column      (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    hcc0m            (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    icei0m           (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    ...               ...
    t0m              (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    t2m              (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    u10m             (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    v10m             (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    vis0m            (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
    xhail0m          (time, y, x) float64 372MB dask.array<chunksize=(20, 589, 789), meta=np.ndarray>
Attributes:
    description:  All prognostic variables for 10-year period on reduced levels

The observations and model data must cover the same timesteps.
Since the example model data hosted on AWS is from 1990 some observations are not available.
For that reason the model data was artificially offset to the year 2020.

In [5]:
datetime_start = np.datetime_as_string(ds_model.time.values[0], unit="s")
datetime_end = np.datetime_as_string(ds_model.time.values[-1], unit="s")
datetime_range = f"{datetime_start}Z/{datetime_end}Z"
print(datetime_range)

2020-09-01T00:00:00Z/2020-09-13T09:00:00Z


In [6]:
dfs = {}

for var in vars_obs:
    params = {
        "api-key": api_key,
        "datetime": datetime_range,
        "parameterId": var,
        "bbox": "7,54,16,58",  # Bounding box for Denmark
        "limit": 100000,
    }

    # Fetch the data
    response = requests.get(url_obs, params=params)
    data = response.json()
    gdf = gpd.GeoDataFrame.from_features(data["features"])
    gdf["time"] = pd.to_datetime(gdf["observed"], utc=True)
    df_pivot = gdf.pivot(index="time", columns="stationId", values="value")
    dfs[var] = df_pivot

In [7]:
# Get the set of stations for each variable
station_sets = [set(dfs[var].columns) for var in vars_obs]

# Find common stations across all variables using set intersection
common_stations = set.intersection(*station_sets)

# Convert to sorted list if needed
common_stations = sorted(list(common_stations))

# Print the number of common stations and their IDs
print(f"Number of stations with data for all variables: {len(common_stations)}")
print("\nStation IDs:")
print(common_stations)

# Filter your DataFrames to keep only common stations
dfs_filtered = {}
for var in vars_obs:
    dfs_filtered[var] = dfs[var][common_stations]
# filter gdf by stations as well
gdf_filtered = gdf[gdf["stationId"].isin(common_stations)]

Number of stations with data for all variables: 48

Station IDs:
['06030', '06032', '06041', '06049', '06052', '06056', '06058', '06060', '06065', '06068', '06070', '06072', '06073', '06074', '06079', '06080', '06081', '06093', '06096', '06102', '06104', '06108', '06110', '06116', '06118', '06119', '06120', '06123', '06124', '06126', '06135', '06138', '06141', '06147', '06149', '06151', '06154', '06156', '06159', '06168', '06169', '06170', '06174', '06180', '06181', '06183', '06188', '06190']


In [8]:
# Convert model times to UTC datetime (assuming ds_model is available)
model_times = pd.to_datetime(ds_model["time"].values).tz_localize("UTC")
# Filter each DataFrame in the dictionary
# Assuming the DataFrames in dfs_filtered already have datetime index
dfs_filtered = {
    k: v[v.index.isin(model_times)] for k, v in dfs_filtered.items()
}
# Filter gdf_filtered
gdf_filtered = gdf_filtered.set_index("time")
gdf_filtered = gdf_filtered[gdf_filtered.index.isin(model_times)]

In [9]:
# Combine the DataFrames into a single xarray Dataset
ds_obs = xr.Dataset(
    {
        var: (["time", "stationId"], dfs_filtered[var].values)
        for var in vars_obs
    },
    coords={
        "time": ds_model.time,
        "stationId": dfs_filtered[vars_obs[0]].columns,
        "lat": (
            "stationId",
            gdf_filtered.groupby("stationId")["geometry"].first().y,
        ),
        "lon": (
            "stationId",
            gdf_filtered.groupby("stationId")["geometry"].first().x,
        ),
    },
)

ds_obs = ds_obs.sel(time=ds_model.time)
ds_obs = ds_obs.sortby("time")
ds_obs

<xarray.Dataset> Size: 194kB
Dimensions:            (time: 100, stationId: 48)
Coordinates:
  * time               (time) datetime64[ns] 800B 2020-09-01 ... 2020-09-13T0...
  * stationId          (stationId) object 384B '06030' '06032' ... '06190'
    lat                (stationId) float64 384B 57.1 57.38 57.74 ... 55.88 55.07
    lon                (stationId) float64 384B 9.851 10.33 ... 12.41 14.75
Data variables:
    wind_speed_past1h  (time, stationId) float64 38kB 1.0 0.3 2.5 ... 3.1 8.2
    wind_dir_past1h    (time, stationId) float64 38kB 62.0 352.0 ... 217.0 239.0
    temp_dry           (time, stationId) float64 38kB 9.1 8.5 12.4 ... 15.5 16.2
    pressure_at_sea    (time, stationId) float64 38kB 1.022e+03 ... 1.022e+03
    pressure           (time, stationId) float64 38kB 1.02e+03 ... 1.02e+03

Conversion of wind speed and direction to u and v.

In [10]:
processor = SynopProcessor()
ds_obs = processor.process_dataset(ds_obs)

Plotting of selected variable

In [11]:
date = ds_obs.isel(time=0)["time"].values
formatted_datetime = pd.to_datetime(date)
formatted_date = formatted_datetime.strftime("%Y-%m-%d")
hour = formatted_datetime.strftime("%H")

Visualization of the observations

In [12]:
ds_obs.isel(time=0)[vars_plot]

<xarray.Dataset> Size: 3kB
Dimensions:          (stationId: 48)
Coordinates:
    time             datetime64[ns] 8B 2020-09-01
  * stationId        (stationId) object 384B '06030' '06032' ... '06188' '06190'
    lat              (stationId) float64 384B 57.1 57.38 57.74 ... 55.88 55.07
    lon              (stationId) float64 384B 9.851 10.33 10.63 ... 12.41 14.75
Data variables:
    u10m             (stationId) float64 384B 0.8829 -0.04175 ... -0.3381 0.4724
    v10m             (stationId) float64 384B 0.4695 0.2971 ... 0.725 2.046
    t2m              (stationId) float64 384B 282.2 281.6 285.5 ... 281.3 nan
    pres_seasurface  (stationId) float64 384B 1.022e+05 1.022e+05 ... 1.019e+05
    pres0m           (stationId) float64 384B 1.02e+05 1.015e+05 ... 1.017e+05